<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC1_Turbofan_FD004_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.svm import LinearSVR

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.exceptions import ConvergenceWarning

from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sns.set_palette('colorblind')

# Data Preparation

In [ ]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [ ]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [ ]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [ ]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(4)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61244,249,251,9.9998,0.2500,100.0,489.05,605.33,1516.36,1315.28,10.52,...,380.16,2388.73,8185.69,8.4541,0.03,372,2319,100.00,29.11,17.5234
61245,249,252,0.0028,0.0015,100.0,518.67,643.42,1598.92,1426.77,14.62,...,535.02,2388.46,8185.47,8.2221,0.03,396,2388,100.00,39.38,23.7151
61246,249,253,0.0029,0.0000,100.0,518.67,643.68,1607.72,1430.56,14.62,...,535.41,2388.48,8193.94,8.2525,0.03,395,2388,100.00,39.78,23.8270
61247,249,254,35.0046,0.8400,100.0,449.44,555.77,1381.29,1148.18,5.48,...,187.92,2388.83,8125.64,9.0515,0.02,337,2223,100.00,15.26,9.0774


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [ ]:
test.shape, y_test.shape

((41214, 26), (248, 1))

In [ ]:
test_last = proc.transform_test_keep_setting(test)
test_last.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,25.0070,0.6214,60.0,462.54,537.66,1264.31,1046.41,7.05,8.99,176.56,...,166.19,2028.53,7890.31,10.7615,0.02,308,1915,84.93,14.41,8.6329
1,41.9989,0.8400,100.0,445.00,549.96,1354.05,1133.55,3.91,5.72,139.03,...,130.17,2387.72,8073.44,9.3925,0.02,331,2212,100.00,10.58,6.4325
2,42.0005,0.8401,100.0,445.00,549.47,1341.06,1118.90,3.91,5.69,139.26,...,130.73,2388.18,8095.58,9.2974,0.02,330,2212,100.00,10.61,6.3488
3,25.0018,0.6207,60.0,462.54,536.06,1253.49,1038.53,7.05,9.00,175.63,...,164.91,2028.30,7878.63,10.8396,0.02,306,1915,84.93,14.41,8.5696
4,25.0039,0.6200,60.0,462.54,537.36,1263.60,1052.52,7.05,9.03,175.53,...,164.95,2028.24,7873.75,10.9094,0.02,307,1915,84.93,14.19,8.6248


In [ ]:
X_test = test_last

### Remaining Useful Life (RUL)

In [ ]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,320
1,1,2,319
2,1,3,318
3,1,4,317
4,1,5,316


## Attributes and target separation

In [ ]:
X_train, y_train = proc.X_y_train_divide_with_settings(train)

In [ ]:
y_train.head()

,RUL
0,320
1,319
2,318
3,317
4,316


In [ ]:
X_train.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,5.70,137.98,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,9.00,174.82,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


## Training and Evaluation functions

In [ ]:
eval = Evaluation()

In [ ]:
search = HyperparameterSearch()

# LinearRegression

## Linear RUL

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.549,RMSE=-60.286
(CV) R2=0.540,RMSE=-60.705


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.233,RMSE=-47.758


## Non-linear RUL

In [ ]:
y_train.clip(upper=189).head()

,RUL
0,189
1,189
2,189
3,189
4,189


In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1)
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 71
n_resources: 500
Fitting 5 folds for each of 71 candidates, totalling 355 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 71 is smaller than n_iter=122. Running 71 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


----------
iter: 1
n_candidates: 24
n_resources: 1500
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 2
n_candidates: 8
n_resources: 4500
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 3
n_candidates: 3
n_resources: 13500
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 106}}
Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fe7c38d8c20>,
                                                                            kw_args={'a_max': 106,
                                                                                     'a_min': 0})))])


Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 106}}
```



In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':106})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fe7c38d8c20>,
                                                                            kw_args={'a_max': 106,
                                                                                     'a_min': 0})))])

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.726,RMSE=-17.404
(CV) R2=0.722,RMSE=-17.512


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.687,RMSE=-20.646


Just adding a non-linear improved results, as in the first set

## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('lin_reg'  ,   LinearRegression())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('lin_reg', LinearRegression())])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 12
n_resources: 500
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 12 is smaller than n_iter=122. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


----------
iter: 1
n_candidates: 4
n_resources: 1500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 2
n_candidates: 2
n_resources: 4500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'poly_ft__interaction_only': False, 'poly_ft__include_bias': True, 'poly_ft__degree': 1}
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1)),
                ('lin_reg', LinearRegression())])


Best Model:
```
PolynomialFeatures(degree=1))
```


In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=1)),
    ('lin_reg'  ,   LinearRegression())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1)),
                ('lin_reg', LinearRegression())])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.549,RMSE=-60.286
(CV) R2=0.540,RMSE=-60.705


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.233,RMSE=-47.758


No clear benefit from polynomial features in linear RUL for simple linear regression. Actually, the degree stayed in 1, i.e., without those features.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1)
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 109}, 'poly_ft__interaction_only': False, 'poly_ft__include_bias': True, 'poly_ft__degree': 1}
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1)),
                ('trf_reg',
                 Transfo

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 109}, 
PolynomialFeatures(degree=1)),
```


In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=1)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':109})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fe7c38d8c20>,
                                                                            kw_args={'a_max': 109,
                                                                                     'a_min': 0})))])

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.726,RMSE=-18.017
(CV) R2=0.722,RMSE=-18.129


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.686,RMSE=-21.234


There was a slight increase in performance when comparing to the scenario without the polynomial features.

# SGD Regressor

The class SGDRegressor implements a plain stochastic gradient descent learning routine which supports different loss functions and penalties to fit linear regression models. **SGDRegressor is well suited for regression problems with a large number of training samples (> 10.000)**, for other problems we recommend Ridge, Lasso, or ElasticNet.

## Linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor())
])

In [ ]:
# We need specific shape 1D arrays for this model
y_train = np.array(y_train).ravel()

In [ ]:
# ~4min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "lin_reg__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "lin_reg__penalty": ['l2', 'l1', 'elasticnet'],
        "lin_reg__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "lin_reg__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__max_iter": [1000,1500,2000,2500,3000],
        "lin_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__learning_rate":['invscaling','constant','optimal','adaptive'],
        "lin_reg__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "lin_reg__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "lin_reg__early_stopping": [True,False],
        "lin_reg__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "lin_reg__n_iter_no_change": [1,2,5,8,10,15,20],
        "lin_reg__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'lin_reg__validation_fraction': 0.4, 'lin_reg__tol': 1e-06, 'lin_reg__power_t': 1, 'lin_reg__penalty': 'elasticnet', 'lin_reg__n_iter_no_change': 2, 'lin_reg__max_iter': 2500, 'lin_reg__loss': 'squared_error', 'lin_reg__learning_rate': 'adaptive', 'lin_reg__eta

Best Model:
```
SGDRegressor(epsilon=2.5, learning_rate='adaptive',
                              max_iter=2500, n_iter_no_change=2,
                              penalty='elasticnet', power_t=1, tol=1e-06,
                              validation_fraction=0.4))
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor(epsilon=2.5, learning_rate='adaptive',
                              max_iter=2500, n_iter_no_change=2,
                              penalty='elasticnet', power_t=1, tol=1e-06,
                              validation_fraction=0.4))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('lin_reg',
                 SGDRegressor(epsilon=2.5, learning_rate='adaptive',
                              max_iter=2500, n_iter_no_change=2,
                              penalty='elasticnet', power_t=1, tol=1e-06,
                              validation_fraction=0.4))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.530,RMSE=-61.579
(CV) R2=0.522,RMSE=-61.827


In [ ]:
X_test = test_last
eval.show_result(y_test, model.predict(X_test))

R2=0.197,RMSE=-48.854


Even poorer results without non-lineariy of RUL. Worse than regular LinearRegressor.

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = SGDRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
# ~5min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "trf_reg__regressor__penalty": ['l2', 'l1', 'elasticnet'],
        "trf_reg__regressor__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "trf_reg__regressor__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__max_iter": [1000,1500,2000,2500,3000],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__learning_rate":['invscaling','constant','optimal','adaptive'],
        "trf_reg__regressor__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "trf_reg__regressor__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "trf_reg__regressor__early_stopping": [True,False],
        "trf_reg__regressor__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "trf_reg__regressor__n_iter_no_change": [1,2,5,8,10,15,20],
        "trf_reg__regressor__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 'trf_reg__regressor__validation_fraction': 0.3, 'trf_reg__regressor__tol': 0.001, 'trf_reg__regressor__power_t': 1.25, 'trf_reg__regressor__penalty': 'l1', 'trf_reg__regressor__n_iter_no_change': 20, 

Best Model:
```
kw_args={'a_max': 135,'a_min': 0}))

regressor=SGDRegressor(alpha=1e-06,
                        average=10,
                        epsilon=1.1,
                        learning_rate='optimal',
                        loss='epsilon_insensitive',
                        n_iter_no_change=20,
                        penalty='l1',
                        power_t=1.25,
                        validation_fraction=0.3),
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor=SGDRegressor(alpha=1e-06,
                                            average=10,
                                            epsilon=1.1,
                                            learning_rate='optimal',
                                            loss='epsilon_insensitive',
                                            n_iter_no_change=20,
                                            penalty='l1',
                                            power_t=1.25,
                                            validation_fraction=0.3),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':135})))
    ])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=SGDRegressor(alpha=1e-06,
                                                                   average=10,
                                                                   epsilon=1.1,
                                                                   learning_rate='optimal',
                                                                   loss='epsilon_insensitive',
                                                                   n_iter_no_change=20,
                                                                   penalty='l1',
                                                                   power_t=1.25,
                                                                   validation_fraction=0.3),
                                            transformer=FunctionTransformer(func=<function c

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.680,RMSE=-25.143
(CV) R2=0.675,RMSE=-25.361


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.647,RMSE=-27.183


Achieved training results from non-linear RUL Linear Regressor, but with a little better test result. Largest clipping across all.

## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor())
])

In [ ]:
# ~25min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "lin_reg__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "lin_reg__penalty": ['l2', 'l1', 'elasticnet'],
        "lin_reg__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "lin_reg__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__max_iter": [1000,1500,2000,2500,3000],
        "lin_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__learning_rate":['invscaling','constant','optimal','adaptive'],
        "lin_reg__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "lin_reg__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "lin_reg__early_stopping": [True,False],
        "lin_reg__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "lin_reg__n_iter_no_change": [1,2,5,8,10,15,20],
        "lin_reg__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions,
                            ignore_warnings=True, scorer='r2')
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'poly_ft__interaction_only': True, 'poly_ft__include_bias': True, 'poly_ft__degree': 3, 'lin_reg__validation_fraction': 0.2, 'lin_reg__tol': 1e-07, 'lin_reg__power_t': 0.5, 'lin_reg__penalty': 'l2', 'lin_reg__n_iter_no_change': 20, 'lin_reg__max_iter': 1000, 'l

Best Model:
```
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, interaction_only=True)),
                ('scaler', StandardScaler()),
                ('lin_reg',
                SGDRegressor(alpha=1e-07, average=2, early_stopping=True,
                            epsilon=1.35, eta0=0.1, learning_rate='constant',
                            loss='epsilon_insensitive', n_iter_no_change=20,
                            power_t=0.5, tol=1e-07,
                            validation_fraction=0.2))
```


In [ ]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scaler', StandardScaler()),
    ('lin_reg',
        SGDRegressor(alpha=1e-07, average=2, early_stopping=True,
                    epsilon=1.35, eta0=0.1, learning_rate='constant',
                    loss='epsilon_insensitive', n_iter_no_change=20,
                    power_t=0.5, tol=1e-07,
                    validation_fraction=0.2))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, interaction_only=True)),
                ('scaler', StandardScaler()),
                ('lin_reg',
                 SGDRegressor(alpha=1e-07, average=2, early_stopping=True,
                              epsilon=1.35, eta0=0.1, learning_rate='constant',
                              loss='epsilon_insensitive', n_iter_no_change=20,
                              power_t=0.5, tol=1e-07,
                              validation_fraction=0.2))])

In [ ]:
# 20min
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.543,RMSE=-60.708
(CV) R2=0.531,RMSE=-61.226


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.262,RMSE=-46.840


No clear benefit from polynomial features in linear RUL as well. It is even worse than the Linear only

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = SGDRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
# ~40min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "trf_reg__regressor__penalty": ['l2', 'l1', 'elasticnet'],
        "trf_reg__regressor__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "trf_reg__regressor__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__max_iter": [1000,1500,2000,2500,3000,4000,4500],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__learning_rate":['invscaling','constant','optimal','adaptive'],
        "trf_reg__regressor__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "trf_reg__regressor__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "trf_reg__regressor__early_stopping": [True,False],
        "trf_reg__regressor__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "trf_reg__regressor__n_iter_no_change": [1,2,5,8,10,15,20],
        "trf_reg__regressor__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                             param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 102}, 'trf_reg__regressor__validation_fraction': 0.1, 'trf_reg__regressor__tol': 0.0001, 'trf_reg__regressor__power_t': 1.25, 'trf_reg__regressor__penalty': 'elasticnet', 'trf_reg__regressor__n_iter_no_chan

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 102}, 
PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
SGDRegressor(epsilon=3,
                eta0=0.1,
                learning_rate='adaptive',
                loss='epsilon_insensitive',
                n_iter_no_change=8,
                penalty='elasticnet',
                power_t=1.25,
                tol=0.0001),
```


In [ ]:
model = Pipeline([
    ('poly_ft',PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
    ('scaler', StandardScaler()),
    ('trf_reg',
        TransformedTargetRegressor(
            check_inverse=False,
            regressor=SGDRegressor(epsilon=3,
                                    eta0=0.1,
                                    learning_rate='adaptive',
                                    loss='epsilon_insensitive',
                                    n_iter_no_change=8,
                                    penalty='elasticnet',
                                    power_t=1.25,
                                    tol=0.0001),
            transformer=FunctionTransformer(np.clip,
                                            kw_args={'a_max': 102,
                                                     'a_min': 0})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=SGDRegressor(epsilon=3,
                                                                   eta0=0.1,
                                                                   learning_rate='adaptive',
                                                                   loss='epsilon_insensitive',
                                                                   n_iter_no_change=8,
                                                                   penalty='elasticnet',
                                                                   power_t=1.25,
                                                                   tol=0.0001),
                          

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.682,RMSE=-17.875
(CV) R2=0.675,RMSE=-18.047


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.618,RMSE=-21.902


With 2-degree terms, we actually decreased performance.

# SVR

We'll use only the linear kernel, because the algorithm is quadratic on the number of samples. Otherwise, the training times get worser.

## Linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('svm_reg'  ,   LinearSVR(random_state=42))
])

In [ ]:
# We need specific shape 1D arrays for this model
y_train = np.array(y_train).ravel()

In [ ]:
# ~15min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "svm_reg__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "svm_reg__fit_intercept": [True, False],
        "svm_reg__max_iter": [1000,3000,5000],
        "svm_reg__epsilon": [1,1e-1,1e-2,1e-3],
        "svm_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "svm_reg__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'svm_reg__tol': 1e-08, 'svm_reg__max_iter': 5000, 'svm_reg__loss': 'squared_epsilon_insensitive', 'svm_reg__fit_intercept': True, 'svm_reg__epsilon': 0.01, 'svm_reg__C': 5, 'scaler': StandardScaler()}
Pipeline(steps=[('scaler', StandardScaler()),
              

Best Model:
```
('scaler', StandardScaler()),
LinearSVR(C=5, epsilon=0.01,
            loss='squared_epsilon_insensitive', max_iter=5000,
            random_state=42, tol=1e-08))
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('svm_reg'  ,   LinearSVR(C=5, epsilon=0.01,
                           loss='squared_epsilon_insensitive', max_iter=5000,
                           random_state=42, tol=1e-08))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_reg',
                 LinearSVR(C=5, epsilon=0.01,
                           loss='squared_epsilon_insensitive', max_iter=5000,
                           random_state=42, tol=1e-08))])

In [ ]:
# ~15min
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


R2=0.546,RMSE=-60.519


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

(CV) R2=0.537,RMSE=-60.892


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.235,RMSE=-47.691


Poor results without non-lineariy of RUL. Comparable to regular LinearRegressor. 

Same model as FD002.

Did not converge

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearSVR(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "trf_reg__regressor__fit_intercept": [True, False],
        "trf_reg__regressor__max_iter": [1000,3000,5000],
        "trf_reg__regressor__epsilon": [1,1e-1,1e-2,1e-3],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "trf_reg__regressor__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 109}, 'trf_reg__regressor__tol': 1e-05, 'trf_reg__regressor__max_iter': 3000, 'trf_reg__regressor__loss': 'squared_epsilon_insensitive', 'trf_reg__regressor__fit_intercept': True, 'trf_reg__regressor__epsil

Best Model:
```
kw_args={'a_max': 109,'a_min': 0}))
('scaler', StandardScaler())
LinearSVR(C=5,
            epsilon=0.01,
            loss='squared_epsilon_insensitive',
            max_iter=3000,
            random_state=42,
            tol=1e-05),
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor=LinearSVR(C=5,
                            epsilon=0.01,
                            loss='squared_epsilon_insensitive',
                            max_iter=3000,
                            random_state=42,
                            tol=1e-05),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':109})))
    ])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearSVR(C=5,
                                                                epsilon=0.01,
                                                                loss='squared_epsilon_insensitive',
                                                                max_iter=3000,
                                                                random_state=42,
                                                                tol=1e-05),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fe7c38d8c20>,
                                                                            kw_args={'a_max': 109,
                                                                                     'a_min': 0})))])

In [ ]:
# ~10min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


R2=0.716,RMSE=-18.339


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

(CV) R2=0.709,RMSE=-18.538


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.682,RMSE=-21.377


Good linear results. But failed to converge.

## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler'   ,   MinMaxScaler()),
    ('svm_reg'  ,   LinearSVR(random_state=42))
])

In [ ]:
# ~2h30min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "svm_reg__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "svm_reg__fit_intercept": [True, False],
        "svm_reg__max_iter": [1000,3000,5000],
        "svm_reg__epsilon": [1,1e-1,1e-2,1e-3],
        "svm_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "svm_reg__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions,
                            ignore_warnings=True, scorer='r2')
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'svm_reg__tol': 1e-05, 'svm_reg__max_iter': 3000, 'svm_reg__loss': 'squared_epsilon_insensitive', 'svm_reg__fit_intercept': True, 'svm_reg__epsilon': 1, 'svm_reg__C': 2, 'scaler': StandardScaler(), 'poly_ft__interaction_only': True, 'poly_ft__include_bias': Fal

Best Model:
```
PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
('scaler', StandardScaler()),
LinearSVR(C=2, epsilon=1, loss='squared_epsilon_insensitive',
                           max_iter=3000, random_state=42, tol=1e-05))
```


In [ ]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler', StandardScaler()),
    ('svm_reg', LinearSVR(C=2, epsilon=1, loss='squared_epsilon_insensitive',
                           max_iter=3000, random_state=42, tol=1e-05))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('scaler', StandardScaler()),
                ('svm_reg',
                 LinearSVR(C=2, epsilon=1, loss='squared_epsilon_insensitive',
                           max_iter=3000, random_state=42, tol=1e-05))])

In [ ]:
# ~1h
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


R2=0.557,RMSE=-59.727


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

(CV) R2=0.537,RMSE=-60.948


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.261,RMSE=-46.887


No benefit from polynomial features in linear RUL.  However, most of the time, it didn't converge, even with 5k iterations. Same model for FD002.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearSVR(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
# ~2h30min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "trf_reg__regressor__fit_intercept": [True, False],
        "trf_reg__regressor__max_iter": [1000,3000,5000],
        "trf_reg__regressor__epsilon": [1,1e-1,1e-2,1e-3],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "trf_reg__regressor__C":[1,2,5,10],
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                             param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 500
max_resources_: 61249
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 122
n_resources: 500
Fitting 5 folds for each of 122 candidates, totalling 610 fits
----------
iter: 1
n_candidates: 41
n_resources: 1500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 2
n_candidates: 14
n_resources: 4500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 3
n_candidates: 5
n_resources: 13500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 4
n_candidates: 2
n_resources: 40500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 128}, 'trf_reg__regressor__tol': 1e-08, 'trf_reg__regressor__max_iter': 3000, 'trf_reg__regressor__loss': 'squared_epsilon_insensitive', 'trf_reg__regressor__fit_intercept': False, 'trf_reg__regressor__epsi

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 128}, 
PolynomialFeatures(degree=3, include_bias=False)),
('scaler', StandardScaler()),
LinearSVR(C=5,
            epsilon=0.01,
            fit_intercept=False,
            loss='squared_epsilon_insensitive',
            max_iter=3000,
            random_state=42,
            tol=1e-08),
```


In [83]:
model = Pipeline([
    ('poly_ft',PolynomialFeatures(degree=3, include_bias=False)),
    ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
    ('scaler', StandardScaler()),
    ('trf_reg',
        TransformedTargetRegressor(
            check_inverse=False,
            regressor=LinearSVR(C=5,
                                epsilon=0.01,
                                fit_intercept=False,
                                loss='squared_epsilon_insensitive',
                                max_iter=3000,
                                random_state=42,
                                tol=1e-08),
            transformer=FunctionTransformer(np.clip,
                                            kw_args={'a_max': 128,
                                                     'a_min': 0})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearSVR(C=5,
                                                                epsilon=0.01,
                                                                fit_intercept=False,
                                                                loss='squared_epsilon_insensitive',
                                                                max_iter=3000,
                                                                random_state=42,
                                                                tol=1e-08),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fe7c38d8c20>

# Continue here

In [ ]:
# ~1h30min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


R2=-5.013,RMSE=-102.548


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

Similar to not using the polynomial features, a little worse. It is also the same model from FD001 and FD002. 